**Apuntes útiles pandas**<br>
- https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python#question3
- https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe
- https://chrisalbon.com/python/data_wrangling/pandas_dropping_column_and_rows/

Se ha establecido un dataframe con la siguiente estructura

|generación|Puerta#1 x| Puerta#1 y|Puerta#2 x| Puerta#2 y|Puerta#3 x| Puerta#3 y|Puerta#4 x| Puerta#4 y|Puerta#5 x| Puerta#5 y|
|-|-|-|-|-|-|-|-|-|-|-|
|..|..|..|..|..|..|..|..|..|..|..|

Donde cada fila corresponderá a un individuo

In [1]:
# https://pynetlogo.readthedocs.io/en/latest/_docs/pynetlogo.html
# https://pynetlogo.readthedocs.io/en/latest/example1.html
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import platform
import pyNetLogo
gui=False
current_gen = 0 #contador de la generación actual

print("inicializando, espere...")
try:
    sistema=platform.system()
    if(sistema=='Windows'):
        netlogo = pyNetLogo.NetLogoLink(gui=False)
    elif(sistema=='Linux'):
        netlogo = pyNetLogo.NetLogoLink(gui=gui,netlogo_home='NetLogo 6.0.4/',netlogo_version='6.0.4')
    else:
        netlogo = pyNetLogo.NetLogoLink(gui=gui)
    print("Listo!")
except:
    print("Ocurrio un problema al levantar netlogo!")

inicializando, espere...
Listo!


In [2]:
df=pd.DataFrame(columns=['gen','x1','y1','x2','y2','x3','y3','x4','y4','x5','y5']) #dataframe que representará a los individuos existentes
ww=netlogo.report("world-width") #obtener el número de columnas de la grilla
wh=netlogo.report("world-height") #obtener el número de filas de la grilla
plan_file = "conference.plan" #archivo plano 
world=list() #representación del mundo en una lista multidimensional (filasXcolumnas)

wall_val=64 #valor de muralla
inner_val=0 #valor espacio interno edificio
outer_Val=2 #valor espacio externo edificio
door_val=45 #valor para las puertas


In [3]:
#abrir el archivo de plano para inicializar la representación de listas multidimensionales
with open(plan_file) as file:
    row=list()
    counter=0
    for i in file:
        patch=i.replace('\n','').split(' ')
        if(counter%ww !=0):
            row.append(int(patch[2]))
        else:
            row=list()
            world.append(row)
        counter = counter + 1
        
#cargar el programa en netlogo
netlogo.load_model(r'escape4_v6.nlogo')
#cargar el plano en netlogo
netlogo.command("load-plan-file")

In [4]:
#simulate recibe la lista ordenada de coordenadas de puertas, realiza la simulación correspondiente y retorna los ticks que ha demorado
#La distribución de puertas debe estar previamente validada, en caso de que las puertas no sean accesibles, la simulación podría quedar atascada en un while infinito
def simulate(doors):
    netlogo.command("set door-data [ [{0} {1}] [{2} {3}] [{4} {5}] [{6} {7}] [{8} {9}] ]".format(*doors))
    netlogo.command("show-plan")
    netlogo.command("generate-population")
    while(netlogo.report("count turtles")>0):
        netlogo.command("go")
    return(netlogo.report("ticks"))

#retorna una lista ordenadas de puertas seleccionadas al azar (válidas)
def pick_random_doors(n):
    #falta implementar
    doors = [0,0,1,1,2,2,3,3,4,4]
    return  doors

#reinicia el dataframe y crea la primera generación, recibe la cantidad de individuos a generar
def genesis(n):
    global df
    df=pd.DataFrame(columns=['gen','x1','y1','x2','y2','x3','y3','x4','y4','x5','y5']) #dataframe que representará a los individuos existentes
    for i in range(n):
        doors_loc=pick_random_doors(5)
        df.loc[df.shape[0]]=[current_gen] + doors_loc    
def catch_the_good_doors(): #Obtener las coordenadas x e y de todas las puertas que nos sirven
    x=netlogo.report("listax")
    y=netlogo.report("listay")
    arr=[]
    for i in range(len(x)):
        arr.append([x[i],y[i]])
    return arr

In [5]:
#ejemplo de simulación para conference.plan
doors=[20,50,40,10,80,15,95,30,60,100]
simulate(doors)


97.0

In [8]:
genesis(10)
df

,gen,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5
0,0,0,0,1,1,2,2,3,3,4,4
1,0,0,0,1,1,2,2,3,3,4,4
2,0,0,0,1,1,2,2,3,3,4,4
3,0,0,0,1,1,2,2,3,3,4,4
4,0,0,0,1,1,2,2,3,3,4,4
5,0,0,0,1,1,2,2,3,3,4,4
6,0,0,0,1,1,2,2,3,3,4,4
7,0,0,0,1,1,2,2,3,3,4,4
8,0,0,0,1,1,2,2,3,3,4,4
9,0,0,0,1,1,2,2,3,3,4,4


In [9]:
l=[1,11,2,2,3,3,4,4,5,5]
s="set door-data [ [{0} {1}] [{2} {3}] [{4} {5}] [{6} {7}] [{8} {9}] ]".format(*l)
s
#ask patches with [pcolor = 64 and (count other patches in-radius 1 with [pcolor = black]) = 1 and (count other patches in-radius 1 with [pcolor = 2]) = 1 ][set pcolor yellow]

'set door-data [ [1 11] [2 2] [3 3] [4 4] [5 5] ]'

In [19]:
netlogo.kill_workspace()

In [8]:
tuplas=catch_the_good_doors() 

In [9]:
tuplas

[[24.0, 37.0],
 [104.0, 22.0],
 [40.0, 62.0],
 [104.0, 50.0],
 [20.0, 41.0],
 [80.0, 62.0],
 [46.0, 37.0],
 [45.0, 7.0],
 [86.0, 62.0],
 [63.0, 62.0],
 [34.0, 27.0],
 [20.0, 42.0],
 [104.0, 17.0],
 [78.0, 11.0],
 [53.0, 32.0],
 [104.0, 56.0],
 [50.0, 62.0],
 [89.0, 4.0],
 [104.0, 54.0],
 [20.0, 44.0],
 [104.0, 24.0],
 [102.0, 4.0],
 [52.0, 27.0],
 [34.0, 62.0],
 [56.0, 21.0],
 [55.0, 62.0],
 [25.0, 13.0],
 [56.0, 13.0],
 [104.0, 60.0],
 [78.0, 21.0],
 [72.0, 22.0],
 [104.0, 32.0],
 [25.0, 26.0],
 [20.0, 43.0],
 [93.0, 62.0],
 [38.0, 37.0],
 [104.0, 25.0],
 [83.0, 62.0],
 [78.0, 17.0],
 [75.0, 22.0],
 [67.0, 62.0],
 [38.0, 62.0],
 [42.0, 62.0],
 [104.0, 48.0],
 [28.0, 37.0],
 [71.0, 62.0],
 [37.0, 62.0],
 [101.0, 4.0],
 [104.0, 12.0],
 [104.0, 10.0],
 [104.0, 33.0],
 [44.0, 27.0],
 [104.0, 57.0],
 [73.0, 62.0],
 [46.0, 27.0],
 [62.0, 62.0],
 [104.0, 45.0],
 [59.0, 22.0],
 [40.0, 27.0],
 [104.0, 8.0],
 [35.0, 62.0],
 [96.0, 62.0],
 [48.0, 62.0],
 [53.0, 36.0],
 [54.0, 7.0],
 [104.0, 20.0